In [1]:
import numpy as np
from numpy.random import RandomState
import matplotlib.pyplot as plt
from scipy import stats

def metropolis_algorithm(uniform_rng, chain_start, n, logtarget, candidate_generating_density):
    x = chain_start
    chain = np.zeros(n)
    accepted = 0
    for i in range(n):
        candidate = candidate_generating_density(x)
        if candidate >= 0:
            if np.log(uniform_rng.uniform()) < logtarget(candidate) - logtarget(x):
                x = candidate
                accepted += 1
        
        chain[i] = x
        
    print(f"Acceptance rate {accepted/n}")
    return chain

Acceptance rate 0.47244


In [2]:
uniform_rng = RandomState(1729)
proposal_rng = RandomState(2)

n = 100000
param = 2
log_exponential = lambda x: np.log(param) - param * x
candidate_generating_density = lambda x: x + 0.60 * proposal_rng.normal()
chain = metropolis_algorithm(uniform_rng, (1/param), n, log_exponential, candidate_generating_density)
chain_after_burn_in = chain[chain.size // 2:]

##########################################################################################
uniform_rng_2 = RandomState(88)
exponential_rvs = -(1 / param) * np.log(1 - uniform_rng_2.rand(chain_after_burn_in.size))
##########################################################################################

fig, ax = plt.subplots(1, 2)
fig.set_figheight(7)
fig.set_figwidth(15)
width = 2
bins = 100
x = np.arange(0, max(chain_after_burn_in) + 0.5, 0.1)

ax[0].hist(chain_after_burn_in, bins, density=True)
ax[0].plot(x, param * np.exp(-param * x), linewidth=width, color="red")
ax[0].set_title("Metropolis-Hastings")

ax[1].hist(exponential_rvs, bins, density=True)
ax[1].plot(x, param * np.exp(-param * x), linewidth=width, color="red")
ax[1].set_title("Inverse transform")

plt.show()

In [3]:
print(stats.anderson(chain_after_burn_in, dist="expon"))
print("--------------------------------------------------")
print(stats.anderson(exponential_rvs, dist="expon"))

AndersonResult(statistic=1.302563736542652, critical_values=array([0.922, 1.078, 1.341, 1.606, 1.957]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))
--------------------------------------------------
AndersonResult(statistic=0.5178271921322448, critical_values=array([0.922, 1.078, 1.341, 1.606, 1.957]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))


In [4]:
np.log(0) < np.log(0.2)

/home/jpssilve/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


True